# Latent Dirichlet Allocation - MCMC implementation

## Sources

1. This notebook directly follows the on the same topic by Andrew Brooks. [link](http://brooksandrew.github.io/simpleblog/articles/latent-dirichlet-allocation-under-the-hood/)
2. The Andrew Brooks tutorial strongly follows some of the original works of topic modeling by [Griffith et. al.](https://webfiles.uci.edu/msteyver/publications/Griffiths_Steyvers_Tenenbaum_2007.pdf)

## Details

While the original tutorial is in R, this code implements the same algorithm in python

In [3]:
import numpy as np

In [83]:
def initialize_topic_model(docs:list,num_topics:int):
    
    docs=[i.split() for i in docs]
    word_map = {word:idx for idx,word in enumerate(list(set([i for j in docs for i in j])))}
    docs=[[word_map[i] for i in j] for j in docs]
    inv_word_map={k:v for v,k in word_map.items()}
    
    word_topic=np.zeros(shape=(num_topics,len(word_map)))
    topic_assignment=[[0 for i in j] for j in docs]
    #topic_assignment=[np.array(i) for i in np.array(topic_assignment,dtype=object)]

    for i in range(len(docs)):
        for j in range(len(docs[i])):
            topic_assignment[i][j]=np.random.randint(low=0,high=num_topics)
            topic_index=topic_assignment[i][j]
            word_index=docs[i][j]
            word_topic[topic_index,word_index]+=1
    doc_topic_count=np.zeros(shape=(len(docs),num_topics))

    for i in range(len(docs)):
        for j in range(num_topics):
            doc_topic_count[i,j]=len([k for k in topic_assignment[i] if k==j])
            
    return docs,word_map,inv_word_map,word_topic,topic_assignment,doc_topic_count


def fit_topic_model(docs:list,num_topics:int,num_iterations:int,eta:float,alpha:float):
    
    docs,word_map,inv_word_map,word_topic,\
    topic_assignment,doc_topic_count = initialize_topic_model(docs,num_topics)
    
    for i in range(num_iterations):
        for d in range(len(docs)):
            for w in range(len(docs[d])):

                topic_prev=topic_assignment[d][w]
                word_id=docs[d][w]

                doc_topic_count[d,topic_prev]-=1
                word_topic[topic_prev,word_id]-=1

                # Update topic assignment
                denom_a = len(docs[d]) + num_topics * alpha
                denom_b = word_topic.sum(axis=1) + len(word_map) * eta

                update_probability=np.divide((word_topic[:,word_id] + eta),denom_b) 
                update_probability+= (doc_topic_count[d,:] + alpha)/denom_a
                update_probability=update_probability/sum(update_probability)

                # sample a word based on new update probability distribution
                topic_new=np.random.choice(list(range(num_topics)),1,p=update_probability/sum(update_probability))[0]

                topic_assignment[d][w]=topic_new
                doc_topic_count[d,topic_new]+=1
                word_topic[topic_new,word_id]+=1

                if topic_prev!=topic_new:
                    print('doc: {}, token: {}\t\t\ttopic {} => {}'.format(d,inv_word_map[word_id],topic_prev,topic_new))

In [82]:
num_topics = 2
alpha = 1
eta = 0.001
iterations = 2
raw_docs=['eat turkey on turkey day holiday',
      'i like to eat cake on holiday',
      'turkey trot race on thanksgiving holiday',
      'snail race the turtle',
      'time travel space race',
      'movie on thanksgiving',
      'movie at air and space museum is cool movie',
      'aspiring movie star']


fit_topic_model(raw_docs,num_topics,iterations,eta,alpha)

doc: 0, token: turkey			topic 0 => 1
doc: 0, token: day			topic 0 => 1
doc: 1, token: i			topic 0 => 1
doc: 1, token: like			topic 1 => 0
doc: 1, token: to			topic 0 => 1
doc: 1, token: eat			topic 1 => 0
doc: 1, token: cake			topic 1 => 0
doc: 2, token: turkey			topic 1 => 0
doc: 2, token: on			topic 0 => 1
doc: 2, token: thanksgiving			topic 1 => 0
doc: 2, token: holiday			topic 1 => 0
doc: 3, token: turtle			topic 1 => 0
doc: 4, token: space			topic 0 => 1
doc: 4, token: race			topic 1 => 0
doc: 5, token: on			topic 0 => 1
doc: 5, token: thanksgiving			topic 1 => 0
doc: 6, token: movie			topic 0 => 1
doc: 6, token: and			topic 1 => 0
doc: 6, token: museum			topic 1 => 0
doc: 7, token: aspiring			topic 1 => 0
doc: 0, token: turkey			topic 0 => 1
doc: 0, token: turkey			topic 1 => 0
doc: 0, token: day			topic 1 => 0
doc: 0, token: holiday			topic 1 => 0
doc: 1, token: i			topic 1 => 0
doc: 1, token: like			topic 0 => 1
doc: 2, token: on			topic 1 => 0
doc: 3, token: race			topic 0 => 